# Chapter 6 and 7 - Loading data, cleaning and preparation

This notebook is light on the various data loading described in Ch 6, because most of those formats described are all just loaded with various pandas `pd.read_...()` helpers that do all the heavy lifting.  Thats great if you have tabular data.  I am instead going to feed you with a netCDF4 file, which takes a little bit more work to load, but only a few lines.  Ping me if you get stuck.

Grab the data here:

https://data.giss.nasa.gov/pub/gistemp/gistemp250_GHCNv4.nc.gz

I'm not sure if the various options for opening that file (described below) handle the compression, so you probably want to `gzip -d` that file.   This dataset has global temperatures, averaged monthly within 2x2 degree boxes over land (the grid points over the oceans are filled with NaN). The data spans from January 1880 through January 2019, though spatial coverage is sparse at the earlier dates and gets much better with time.

The imports below should cover the various ways to complete the notebook.  

##### Reference

GISTEMP Team, 2019: GISS Surface Temperature Analysis (GISTEMP v4). NASA Goddard Institute for Space Studies. Dataset accessed 2019-03-06 at https://data.giss.nasa.gov/gistemp/.

Hansen, J., R. Ruedy, M. Sato, and K. Lo, 2010: Global surface temperature change, Rev. Geophys., 48, RG4004, doi:10.1029/2010RG000345.

In [1]:
# You may not need all of these, but you shouldn't need more. YOLO
import pandas as pd
import netCDF4 as nc4
import xarray as xr
import seaborn as sns
import numpy as np
sns.set()
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [5]:
datafile_nc4 =  "/Users/nswarr/Downloads/gistemp250_GHCNv4.nc"

### Working with a netCDF4 data file

There are a few ways to go about this, as in anything python related.  Pandas does not speak netCDF by default, but there are some easy ways to get this data into a dataframe.

1. Load the data with the netCDF4 package and build a dataframe with the variables you are interested in

In [111]:
raw_data = nc4.Dataset(datafile_nc4, "r", format="NETCDF4")

In [16]:
raw_data.dimensions

OrderedDict([('lat',
              <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 90),
             ('lon',
              <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 180),
             ('time',
              <class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 1669),
             ('nv',
              <class 'netCDF4._netCDF4.Dimension'>: name = 'nv', size = 2)])

In [17]:
lat = raw_data.dimensions['lat']

In [21]:
lat_group = lat.group()

In [22]:
lat_group

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    title: GISTEMP Surface Temperature Analysis
    institution: NASA Goddard Institute for Space Studies
    source: http://data.giss.nasa.gov/gistemp/
    Conventions: CF-1.6
    history: Created 2019-02-14 20:47:03 by SBBX_to_nc 2.0 - ILAND=250,  IOCEAN=none,     Base: 1951-1980
    dimensions(sizes): lat(90), lon(180), time(1669), nv(2)
    variables(dimensions): float32 lat(lat), float32 lon(lon), int32 time(time), int32 time_bnds(time,nv), int16 tempanomaly(time,lat,lon)
    groups: 

In [25]:
raw_data.variables

OrderedDict([('lat', <class 'netCDF4._netCDF4.Variable'>
              float32 lat(lat)
                  standard_name: latitude
                  long_name: Latitude
                  units: degrees_north
              unlimited dimensions: 
              current shape = (90,)
              filling on, default _FillValue of 9.969209968386869e+36 used),
             ('lon', <class 'netCDF4._netCDF4.Variable'>
              float32 lon(lon)
                  standard_name: longitude
                  long_name: Longitude
                  units: degrees_east
              unlimited dimensions: 
              current shape = (180,)
              filling on, default _FillValue of 9.969209968386869e+36 used),
             ('time', <class 'netCDF4._netCDF4.Variable'>
              int32 time(time)
                  long_name: time
                  units: days since 1800-01-01 00:00:00
                  bounds: time_bnds
              unlimited dimensions: 
              current shape = (1

In [37]:
lat = raw_data.variables['lat'][:]

In [38]:
lat

masked_array(data=[-89., -87., -85., -83., -81., -79., -77., -75., -73.,
                   -71., -69., -67., -65., -63., -61., -59., -57., -55.,
                   -53., -51., -49., -47., -45., -43., -41., -39., -37.,
                   -35., -33., -31., -29., -27., -25., -23., -21., -19.,
                   -17., -15., -13., -11.,  -9.,  -7.,  -5.,  -3.,  -1.,
                     1.,   3.,   5.,   7.,   9.,  11.,  13.,  15.,  17.,
                    19.,  21.,  23.,  25.,  27.,  29.,  31.,  33.,  35.,
                    37.,  39.,  41.,  43.,  45.,  47.,  49.,  51.,  53.,
                    55.,  57.,  59.,  61.,  63.,  65.,  67.,  69.,  71.,
                    73.,  75.,  77.,  79.,  81.,  83.,  85.,  87.,  89.],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [39]:
raw_data.variables['lon'][:]

masked_array(data=[-179., -177., -175., -173., -171., -169., -167., -165.,
                   -163., -161., -159., -157., -155., -153., -151., -149.,
                   -147., -145., -143., -141., -139., -137., -135., -133.,
                   -131., -129., -127., -125., -123., -121., -119., -117.,
                   -115., -113., -111., -109., -107., -105., -103., -101.,
                    -99.,  -97.,  -95.,  -93.,  -91.,  -89.,  -87.,  -85.,
                    -83.,  -81.,  -79.,  -77.,  -75.,  -73.,  -71.,  -69.,
                    -67.,  -65.,  -63.,  -61.,  -59.,  -57.,  -55.,  -53.,
                    -51.,  -49.,  -47.,  -45.,  -43.,  -41.,  -39.,  -37.,
                    -35.,  -33.,  -31.,  -29.,  -27.,  -25.,  -23.,  -21.,
                    -19.,  -17.,  -15.,  -13.,  -11.,   -9.,   -7.,   -5.,
                     -3.,   -1.,    1.,    3.,    5.,    7.,    9.,   11.,
                     13.,   15.,   17.,   19.,   21.,   23.,   25.,   27.,
                     29.,

In [53]:
raw_data.variables['time'][:]

masked_array(data=[29233, 29264, 29293, ..., 79941, 79971, 80002],
             mask=False,
       fill_value=999999,
            dtype=int32)

In [40]:
raw_data.variables['tempanomaly'][:]

masked_array(
  data=[[[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         ...,
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --]],

        [[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         ...,
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --]],

        [[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         ...,
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --]],

        ...,

        [[0.41999998688697815, 0.41999998688697815, 0.41999998688697815,
          ..., 0.41999998688697815, 0.41999998688697815,
          0.41999998688697815],
         [0.41999998688697815, 0.419999986

2. Load the data with xarray.open_dataset and convert that to a dataframe with to_dataframe(). 

There is more in this file than we really need, so if you can get to this state, you win:

```
>>> print(df.columns)

Index(['lat', 'lon', 'time', 'tempanomaly'], dtype='object')
```

In [32]:
ds = xr.open_dataset(datafile_nc4)
df = ds.to_dataframe()

In [35]:
print(df.columns)

Index(['time_bnds', 'tempanomaly'], dtype='object')


In [118]:
print(df.head())

                            time_bnds  tempanomaly
lat   lon    nv time                              
-89.0 -179.0 0  1880-01-15 1880-01-01          NaN
                1880-02-15 1880-02-01          NaN
                1880-03-15 1880-03-01          NaN
                1880-04-15 1880-04-01          NaN
                1880-05-15 1880-05-01          NaN


In [112]:
the_index = df.index

In [113]:
the_index.names

FrozenList(['lat', 'lon', 'nv', 'time'])

In [36]:
df.describe()

,tempanomaly
count,1.711152e+07
mean,2.098133e-01
std,2.012221e+00
min,-1.989000e+01
25%,-7.300000e-01
50%,1.600000e-01
75%,1.100000e+00
max,2.048000e+01


In [55]:
row = df.iloc[1]
row

time_bnds      1880-02-01 00:00:00
tempanomaly                    NaN
Name: (-89.0, -179.0, 0, 1880-02-15 00:00:00), dtype: object

In [62]:
row.name

(-89.0, -179.0, 0, Timestamp('1880-02-15 00:00:00'))

In [63]:
row.time_bnds

Timestamp('1880-02-01 00:00:00')

In [67]:
df2 = df.reset_index()
df2

,lat,lon,nv,time,time_bnds,tempanomaly
0,-89.0,-179.0,0,1880-01-15,1880-01-01,NaN
1,-89.0,-179.0,0,1880-02-15,1880-02-01,NaN
2,-89.0,-179.0,0,1880-03-15,1880-03-01,NaN
3,-89.0,-179.0,0,1880-04-15,1880-04-01,NaN
4,-89.0,-179.0,0,1880-05-15,1880-05-01,NaN
5,-89.0,-179.0,0,1880-06-15,1880-06-01,NaN
6,-89.0,-179.0,0,1880-07-15,1880-07-01,NaN
7,-89.0,-179.0,0,1880-08-15,1880-08-01,NaN
8,-89.0,-179.0,0,1880-09-15,1880-09-01,NaN
9,-89.0,-179.0,0,1880-10-15,1880-10-01,NaN


In [ ]:
col = df.columns

In [76]:
df2.columns

Index(['lat', 'lon', 'nv', 'time', 'time_bnds', 'tempanomaly'], dtype='object')

In [82]:
df2 = df2.set_index('time')
df2

,lat,lon,nv,time_bnds,tempanomaly
time,,,,,
1880-01-15,-89.0,-179.0,0,1880-01-01,NaN
1880-02-15,-89.0,-179.0,0,1880-02-01,NaN
1880-03-15,-89.0,-179.0,0,1880-03-01,NaN
1880-04-15,-89.0,-179.0,0,1880-04-01,NaN
1880-05-15,-89.0,-179.0,0,1880-05-01,NaN
1880-06-15,-89.0,-179.0,0,1880-06-01,NaN
1880-07-15,-89.0,-179.0,0,1880-07-01,NaN
1880-08-15,-89.0,-179.0,0,1880-08-01,NaN
1880-09-15,-89.0,-179.0,0,1880-09-01,NaN


The lat and lon variables give global coverage every 2 degrees (only odd value of lat and lon) of temperature anomaly (the deviation from the climate average).  Produce a timeseries (data or plot) of the temperature anomaly near Boston, MA. (lat,lon = 43.0, -71.0)

In [102]:
temp_near_boston = df2[(df2.lat == 43.0) & (df2.lon == -71.0)]
temp_near_boston

,lat,lon,nv,time_bnds,tempanomaly
time,,,,,
1880-01-15,43.0,-71.0,0,1880-01-01,3.60
1880-02-15,43.0,-71.0,0,1880-02-01,1.22
1880-03-15,43.0,-71.0,0,1880-03-01,-2.04
1880-04-15,43.0,-71.0,0,1880-04-01,-0.47
1880-05-15,43.0,-71.0,0,1880-05-01,2.94
1880-06-15,43.0,-71.0,0,1880-06-01,0.36
1880-07-15,43.0,-71.0,0,1880-07-01,-0.65
1880-08-15,43.0,-71.0,0,1880-08-01,-0.95
1880-09-15,43.0,-71.0,0,1880-09-01,0.41


Are there any NaN or missing values in this data?  

In [121]:
len(temp_near_boston[temp_near_boston.tempanomaly.isnull()])

0

In [122]:
temp_near_boston.tempanomaly.isnull().values.any()

False

Now take a look at the same data at the location (lat,lon = 1.0, -87.0).  Does this data have missing values or NaN?  Are they appropriate? 

In [124]:
pacific_ocean = df2[(df2.lat == 1.0) & (df2.lon == -87.0)]

print(len(pacific_ocean.tempanomaly.isnull()))
print(pacific_ocean.head())

3338
            lat   lon  nv  time_bnds  tempanomaly
time                                             
1880-01-15  1.0 -87.0   0 1880-01-01          NaN
1880-02-15  1.0 -87.0   0 1880-02-01          NaN
1880-03-15  1.0 -87.0   0 1880-03-01          NaN
1880-04-15  1.0 -87.0   0 1880-04-01          NaN
1880-05-15  1.0 -87.0   0 1880-05-01          NaN


Now let's focus on a slice of this data in time.  Pick a date in 2018 and all the temperature anomalies and locations associated with that date.  This data is going to come out of pandas, most likely, looking like this:

```
           lat    lon  tempanomaly
1666     -89.0 -179.0         0.42
3335     -89.0 -179.0         0.42
5004     -89.0 -177.0         0.42
...
[32400 rows x 3 columns]
```

(your data will be different if you chose a different date)

Find the average of the warmest and coolest 10% of the data for the date you chose.

In [107]:
df2.index

DatetimeIndex(['1880-01-15', '1880-02-15', '1880-03-15', '1880-04-15',
               '1880-05-15', '1880-06-15', '1880-07-15', '1880-08-15',
               '1880-09-15', '1880-10-15',
               ...
               '2018-04-15', '2018-05-15', '2018-06-15', '2018-07-15',
               '2018-08-15', '2018-09-15', '2018-10-15', '2018-11-15',
               '2018-12-15', '2019-01-15'],
              dtype='datetime64[ns]', name='time', length=54075600, freq=None)

In [110]:
df2[df2.index.year == 2018].tempanomaly.isna().count()

388800

#### BONUS!  (I solved this with stuff that turns out was in CH8, but I'm keeping it in)

The last question had data like this:

```
           lat    lon  tempanomaly
1666     -89.0 -179.0         0.42
3335     -89.0 -179.0         0.42
5004     -89.0 -177.0         0.42
...
[32400 rows x 3 columns]
```

I'm going give you some code to plot a heatmap, but first the data needs to be transformed to look like this:

```
lon    -179.0  -177.0  -175.0  -173.0  -171.0  -169.0  -167.0  -165.0  -163.0  \
lat                                                                             
 81.0     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
 79.0     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
 77.0     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
 ...
[86 rows x 180 columns]
```

Go for it!

If your data looks like the 86x180 2-d example above, then go ahead and run this next cell.  You'll need to change the globaltemps, of course, to whatever your dataset is called.  If you chocse "2018-11-15" as your date, your output should look like this:

![image.png](image.png)

In [ ]:
sns.heatmap(data=globaltemps, center=0, cmap="bwr")

This plot looks vaguely like a world map, because this dataset only has values for land. 

Congrats, you're done.  I was asked for warm weather, and I think this dataset delivers.